# STA 141B WQ 25 Homework Assignment 2

## Instructions

- Complete the exercises below. Create more code chunks if necessary. Answer all questions. Show results for both the *test* and *run* cases.
- Export the Jupyter Notebook as an PDF file.
- Submit the PDF by **Sunday, February 9, at 11:59 PM PT** to Gradescope.
- For each exercise, indicate the region of your answer in the PDF to facilitate grading. 

## Additional information

- Complete this worksheet yourself. 
- You may use the internet or discuss possible approaches to solve the problems with other students. You are not allowed to share your code or your answers with other students.
- No other libraries than those explicitly allowed can be used. 
- Use code cells for your Python scripts and Markdown cells for explanatory text or answers to non-coding questions. Answer all textual questions in complete sentences.
- Late homework submissions will not be accepted. No submissions will be accepted by email.
- The total number of points for this assignment is 20.

__Exercise 1__

In this assignment, you'll use SQL queries to explore data in Lahman's Baseball Database, which contains "complete batting and pitching statistics from 1871 to 2020, plus fielding statistics, standings, team stats, managerial records, post-season data, and more."

We use the 2019 version for this homework. You can find the database in SQLite format on Piazza.

Documentation for the database, including a description of all tables, is in the `readme2019.txt` file. 

__(a)__ From 1990 to 1999, which team had the highest payroll? Payroll means the sum of the salaries for all people on the team in a given year. Return a `pandas.DataFrame` with column names `year`, `payroll` and `team`, which is the team name. 

__Test:__

```
    year	payroll		team
	2000	92338260.0	 New York Yankees
	2001	112287143.0	New York Yankees
	2002	125928583.0	New York Yankees
	2003	152749814.0	New York Yankees
```

__(b, i)__ For each year between 1990 and 1999, identify the player with the highest salary. 

__Test:__ 

```
	 year    name		 salary
 	2000	Brown		15714286.0
 	2001	Rodriguez	22000000.0
 	2002	Rodriguez	22000000.0
 	2003	Rodriguez	22000000.0
 	2004	Ramirez	  22500000.0
```

__(b, ii)__ For these players, obtain their log-salary over their entire career. _Print the length of the data set of their career log-salaries_. 

__Test:__ For example, the career salary for for Kevin Brown, the player with the largest salary in 2000, is given by: 

```
	year	name	 salary
	1989	Brown	72500.0
	1990	Brown	218000.0
	1991	Brown	355000.0
	1992	Brown	1200000.0
	1993	Brown	2800000.0
	1994	Brown	4225000.0
	1995	Brown	4225000.0
	1996	Brown	3350000.0
	1997	Brown	4510000.0
	1998	Brown	4935000.0
	1999	Brown	10714286.0
	2000	Brown	15714286.0
	2001	Brown	15714286.0
	2002	Brown	15714286.0
	2003	Brown	15714286.0
	2004	Brown	15714286.0
	2005	Brown	15714286.0
```

The length of the data frame returned for the career salaries of the best-paid players in 2000-2004 is given as 

```
> data = pd.read_sql("...")
> data.shape[0]
58
```

__(b, iii)__ Plot the career salaries over time, colored by player. Add a scatter point for the year each player had the largest salary. 

__Test:__
<img src="./source/test.png" alt="drawing" width="500"/>

__Exercise 2__

We will use the [lichess](https://lichess.org/api) API to retrieve some information about the current state of chess in the world. In order to answer below questions, make precise and economical requests. You may use:

```
import requests
import json import loads
import pandas

from datetime import datetime
```

__(a)__ What is the real name of the player `muisback`? 

In [2]:
import requests
import json
from datetime import datetime

In [3]:
website = 'https://lichess.org'

username = 'muisback'
apiPath = f'/api/user/{username}'

url = f"{website}{apiPath}"
response = requests.get(url)
res_json = response.json()

In [4]:
'Real Name: ' + res_json['profile']['realName']

'Real Name: Rauf Mamedov'

In [5]:
res_json['createdAt']

1580238632291

In [6]:
f'Created At: {datetime.fromtimestamp(res_json['createdAt'] / 1000)}'

'Created At: 2020-01-28 11:10:32.291000'

__(b, i)__ Get the username of the last player that played a rapid game against user `athena-pallada` in 2023. __(ii)__ In all games against this user, what is the win-to-loss ratio of `athena-pallada`?

*Hints: For (i), use [this](https://www.epochconverter.com/) to obtain your epoch time. Bor both (i) and (ii): What kind of data type to you accept in your header? For (iii): Why can't the JSON be converted to a dict? Inspect the string to find the cause of the error. Use string methods to fix the problem.*

In [7]:
# i
website = 'https://lichess.org'
username = 'athena-pallada'
apiPath = f'/api/games/user/{username}'

queryParams = {
  'perfType': 'rapid',
  'since': int(datetime(2023, 1, 1).timestamp() * 1000),
  'until': int(datetime(2023, 12, 31).timestamp() * 1000),
  'sort': 'dateDesc',
  'max': 1,
}

headers = {
  "Accept": "application/x-ndjson"
}

url = f'{website}{apiPath}?{'&'.join(f'{key}={value}' for key, value in queryParams.items())}'

response = requests.get(url, headers=headers)
response.status_code

200

In [8]:
res_json = response.json()

In [9]:
players = res_json['players']

In [10]:
print(json.dumps(players, indent=2)) # the last player that played against athena-pallada is debilych

{
  "white": {
    "user": {
      "name": "athena-pallada",
      "title": "GM",
      "flair": "nature.fire",
      "id": "athena-pallada"
    },
    "rating": 2837
  },
  "black": {
    "user": {
      "name": "Debilych",
      "flair": "activity.performing-arts",
      "id": "debilych"
    },
    "rating": 1500,
    "provisional": true
  }
}


In [7]:
# ii
website = 'https://lichess.org'
username = 'athena-pallada'
apiPath = f'/api/games/user/{username}'

queryParams = {
  'vs': 'debilych',
}

url = f'{website}{apiPath}?{'&'.join(f'{key}={value}' for key, value in queryParams.items())}'

headers = {
  "Accept": "application/x-ndjson"
}

response = requests.get(url, headers=headers)
response.status_code

200

In [8]:
res_json = [json.loads(json_str) for json_str in response.text.strip().split('\n')]

In [9]:
res_json

[{'id': 'RZN3Y8nt',
  'rated': False,
  'variant': 'standard',
  'speed': 'rapid',
  'perf': 'rapid',
  'createdAt': 1701020082614,
  'lastMoveAt': 1701020175436,
  'status': 'outoftime',
  'source': 'simul',
  'players': {'white': {'user': {'name': 'athena-pallada',
     'title': 'GM',
     'flair': 'nature.fire',
     'id': 'athena-pallada'},
    'rating': 2837},
   'black': {'user': {'name': 'Debilych',
     'flair': 'activity.performing-arts',
     'id': 'debilych'},
    'rating': 1500,
    'provisional': True}},
  'winner': 'white',
  'moves': 'b3',
  'clock': {'initial': 900, 'increment': 0, 'totalTime': 900}},
 {'id': 'ocN7AmdZ',
  'rated': False,
  'variant': 'standard',
  'speed': 'rapid',
  'perf': 'rapid',
  'createdAt': 1701018914984,
  'lastMoveAt': 1701023147011,
  'status': 'resign',
  'source': 'simul',
  'players': {'white': {'user': {'name': 'athena-pallada',
     'title': 'GM',
     'flair': 'nature.fire',
     'id': 'athena-pallada'},
    'rating': 2837},
   'black'

In [10]:
def did_win(player_id, game):
  is_white = game['players']['white']['user']['id'] == player_id
  winner_is_white = game['winner'] == 'white'
  return is_white and winner_is_white or not is_white and not winner_is_white

did_win_games = [did_win(username, game) for game in res_json]

In [12]:
did_win_games

[True, False]

In [11]:
win_ratio = sum(did_win_games) / sum([not win for win in did_win_games])
win_ratio

1.0

__(c)__ I want to learn about the [December Arena Tournament](https://lichess.org/tournament/dec24lta). __(i)__ How many games have been played in total? __(ii)__ How many games have been played by the winner? __(iii)__ How many players played in the tournament? __(iv)__ Which players played exactly 22 games?

__(d, i)__ Obtain all rated and analyzed ultrabullet games that player `nihalsarin2004` played in December 2024. How many games do you find? __(ii)__ From each game, extract the information whether `nihalsarin2004` played black or white, which color won, whether he played against a player with a higher rating or not and what opening was played (ignore variations). What proportion of games did `nihalsarin2004` win? What proportion of games did he win as white? What percentage of opponents were stronger than `nihalsarin2004`? 